# Congress.gov API in Python

by Sebastian Shirk

**Congress.gov:** https://www.congress.gov

**API Documentation:** https://api.congress.gov

**API GitHub:** https://github.com/LibraryOfCongress/api.congress.gov/

**Reuse Policy and More Info:** https://www.loc.gov/legal

*These recipe examples were tested on August 27, 2024.*

## Setup

### Import Libraries

In [12]:
import requests
import time
from pprint import pprint

### API Key

For this API you need an API key. You can get one by signing up for free at https://api.data.gov/signup/.<br>\
Save your API key in a file called `api_key.py` and import your key. We will also use a `BASE_URL` constant for the API.

In [2]:
from api_key import API_KEY
BASE_URL = 'https://api.congress.gov/v3/'

## 1. Retrieving Bill Data

### Get Recent Bills

Get a list of recent bills as well as their provided information. 

In [28]:
def get_recent_bills(limit=2):
    print("Recent Bills:")
    endpoint = f'bill?api_key={API_KEY}&format=json&limit={limit}'
    response = requests.get(BASE_URL + endpoint)
    if response.status_code == 200:
        bills = response.json()['bills']
        for bill in bills:
            print(f"Title: {bill['title']}")
            print(f"Text: {bill['latestAction']['text']}")
            print(f"Origin Chamber: {bill['originChamber']}")
            print(f"Action Date: {bill['latestAction']['actionDate']}")
            print(f"Update Date: {bill['updateDate']}")
            print(f"Number: {bill['number']}")
            print(f"Bill Type: {bill['type']}")
            print()
    else:
        print("Failed to fetch recent bills.")

# get_recent_bills(limit=num)
# Limit is optional, default is 2 with a max of 250
get_recent_bills()

Recent Bills:
Title: DETECT Fentanyl and Xylazine Act of 2024
Text: Placed on the Union Calendar, Calendar No. 531.
Origin Chamber: House
Action Date: 2024-08-23
Update Date: 2024-08-24
Number: 8663
Bill Type: HR

Title: Decoupling from Foreign Adversarial Battery Dependence Act
Text: Placed on the Union Calendar, Calendar No. 530.
Origin Chamber: House
Action Date: 2024-08-23
Update Date: 2024-08-24
Number: 8631
Bill Type: HR



### Get Number of Bills in a Specified Time Frame

Gather the number of bills that were introduced in a specified time frame as well as how many became public law.

This may take a while to run when spanning over certain months, so be prepared to wait if you run this block.

In [29]:
def get_bills(start_date, end_date, offset=0):
    endpoint = f'bill?api_key={API_KEY}&format=json&fromDateTime={start_date}&toDateTime={end_date}&limit=250&offset={offset}'
    response = requests.get(BASE_URL + endpoint)
    if response.status_code == 200:
        return response.json().get('bills', [])
    else:
        print(f"Failed to fetch bills. Status code: {response.status_code}")
        return []

def fetch_all_bills(start_date, end_date):
    all_bills = []
    offset = 0 
    max_requests = 10000
    request_count = 0
    
    while request_count < max_requests:
        bills = get_bills(start_date, end_date, offset)
        year = start_date.split('-')[0]
        month = start_date.split('-')[1]
        if not bills:
            break

        filtered_bills = []
        for bill in bills:
            action_date = bill.get('updateDate', {})
            if (year not in action_date):
                continue
            elif not bill['latestAction']['text'].lower().startswith('became public law'):
                filtered_bills.append(bill)
                continue
            else:
                filtered_bills.append(bill)
        
        all_bills.extend(filtered_bills)
        
        if len(bills) < 250:
            break
        
        offset += 250
        request_count += 1
        print(f"Offset: {offset}, Total Bills Collected: {len(all_bills)}, Total Bills Passed: {sum(1 for bill in all_bills if 'latestAction' in bill and bill['latestAction']['text'].lower().startswith('became public law'))}")
        time.sleep(1)
    
    if request_count >= max_requests:
        print(f"Reached the maximum number of requests ({max_requests}). Stopping further requests.")
    
    return all_bills

def count_bills(bills_data):
    total_bills = len(bills_data)
    passed_bills = sum(1 for bill in bills_data if 'latestAction' in bill and bill['latestAction']['text'].lower().startswith('became public law'))
    return total_bills, passed_bills

start_date = "2023-2-01T00%3A00%3A00Z"
end_date = "2023-2-28T23%3A59%3A59Z"
print(f"Fetching bills for {start_date} to {end_date}")
bills_data = fetch_all_bills(start_date, end_date)
total_bills, passed_bills = count_bills(bills_data)
print(f"Total bills: {total_bills}")
print(f"Passed bills: {passed_bills}")

Fetching bills for 2023-2-01T00%3A00%3A00Z to 2023-2-28T23%3A59%3A59Z
Total bills: 106
Passed bills: 68


### Get Summaries of Recent Bills

Get all information on given bills as well as their summaries. Summaries are given in HTML format.

In [43]:
def get_recent_summaries(limit=2):
    endpoint = f'summaries?api_key={API_KEY}&format=json&limit={limit}'
    response = requests.get(BASE_URL + endpoint)
    if response.status_code == 200:
        summaries = response.json()['summaries']
        for summary in summaries:
            print(f"Action Date: {summary['actionDate']}")
            pprint(summary['bill'])
            print("HTML output:")
            pprint(summary['text'])
            print()
    else:
        print("Failed to fetch recent summaries.")

# get_recent_summaries(limit=num)
# Limit is optional, default is 2 with a max of 250
get_recent_summaries()

Action Date: 2023-06-06
{'congress': 118,
 'number': '1835',
 'originChamber': 'Senate',
 'originChamberCode': 'S',
 'title': 'National Cybersecurity Awareness Act',
 'type': 'S',
 'updateDateIncludingText': '2024-08-27T20:23:11Z',
 'url': 'https://api.congress.gov/v3/bill/118/s/1835?format=json'}
HTML output:
('<p><strong>National Cybersecurity Awareness Act</strong></p><p>This bill '
 'requires the Cybersecurity and Infrastructure Security Agency (CISA) of the '
 'Department of Homeland Security to lead and coordinate federal efforts to '
 'promote national cybersecurity awareness and to establish a program for '
 'planning and coordinating federal cybersecurity awareness '
 'campaigns.</p><p>CISA must also inform nonfederal entities of voluntary '
 'cyber hygiene best practices, including information on how to prevent '
 'cyberattacks and mitigate cybersecurity risks.</p><p>Further, CISA shall '
 'consult with private sector entities, state, local, tribal, and territorial '
 'govern

### Get Recent Amemdments

Get a list of recent amendments as well as their provided information.

In [48]:
def get_recent_amendments(limit=5):
    endpoint = f'amendment?api_key={API_KEY}&format=json&limit={limit}'
    response = requests.get(BASE_URL + endpoint)
    if response.status_code == 200:
        amendments = response.json()['amendments']
        for amendment in amendments:
            if 'latestAction' not in amendment:
                continue
            print(f"Action Date: {amendment['latestAction']['actionDate']}")
            print(f"Text: {amendment['latestAction']['text']}")
            if 'purpose' in amendment:
                print(f"Purpose: {amendment['purpose']}")
            print(f"Number: {amendment['number']}")
            print(f"Type: {amendment['type']}")
            print()
    else:
        print("Failed to fetch recent amendments.")

print("Recent Amendments:")
# get_recent_amendments(limit=num)
# Limit is optional, default is 5 with a max of 250
get_recent_amendments()

Recent Amendments:
Action Date: 2024-08-01
Text: Amendment SA 3224 agreed to in Senate by Unanimous Consent.
Purpose: In the nature of a substitute.
Number: 3224
Type: SAMDT



## 2. Retrieving Treaty Data

### Get Recent Treaties

Get a list of recent treaties as well as their provided information.

We'll use the "Number" to get more information on a specific treaty in the next cell.

In [49]:
def get_recent_treaties(limit=2):
    endpoint = f'treaty?api_key={API_KEY}&format=json&limit={limit}'
    response = requests.get(BASE_URL + endpoint)
    if response.status_code == 200:
        treaties = response.json()['treaties']
        for treaty in treaties:
            print(f"Topic: {treaty['topic']}")
            print(f"Transmitted Date: {treaty['transmittedDate']}")
            print(f"Update Date: {treaty['updateDate']}")
            print(f"Congress Received: {treaty['congressReceived']}")
            print(f"Congress Considered: {treaty['congressConsidered']}")
            print(f"Number: {treaty['number']}")
            print()
    else:
        print("Failed to fetch recent treaties.")

print("Recent Treaties:")
# get_recent_treaties(limit=num)
# Limit is optional, default is 2 with a max of 250
get_recent_treaties()

Recent Treaties:
Topic: None
Transmitted Date: None
Update Date: 2024-04-17T14:13:22Z
Congress Received: 117
Congress Considered: None
Number: 7

Topic: Extradition
Transmitted Date: 2017-01-17T00:00:00Z
Update Date: 2024-03-08T09:49:30Z
Congress Received: 115
Congress Considered: 115
Number: 1



### Get Detailed Information on Treaties

Get detailed information on a specific treaty by using the "Number" from the previous cell.

Requires a specific treaty number and a specific congress number.

In [51]:
def get_treaty_details(congress_numbers):
    treaty_number = 1
    for congress in congress_numbers:
        while True:
            endpoint = f'treaty/{congress}/{treaty_number}?api_key={API_KEY}&format=json'
            response = requests.get(BASE_URL + endpoint)
            if response.status_code == 200:
                treaty = response.json()['treaty']
                print(f"Congress: {congress}")
                print(f"Treaty Number: {treaty_number}")
                print(f"Topic: {treaty['topic']}")
                print(f"Index Terms: {treaty['indexTerms']}")
                print(f"Number: {treaty['number']}")
                pprint(f"Resolution Text: {treaty['resolutionText']}")
                pprint(f"Title: {treaty['titles'][0]}")
                print(f"Transmitted Date: {treaty['transmittedDate']}")
                print(f"Update Date: {treaty['updateDate']}")
                print()
                treaty_number += 1
            else:
                print(f"No more treaties found for Congress {congress}.")
                print()
                break
        treaty_number = 1

print("Treaty Details:")
get_treaty_details(['117', '118'])

Treaty Details:
Congress: 117
Treaty Number: 1
Topic: International Law and Organization
Index Terms: [{'name': '117-1'}, {'name': 'Kigali'}, {'name': 'Kigali Amendment'}, {'name': 'Montreal'}, {'name': 'Montreal Protocol'}, {'name': 'amendment'}, {'name': 'ozone'}]
Number: 1
('Resolution Text: <p>As approved by the Senate:</p><br><p> '
 '</p><br><p>Resolved (two-thirds of the Senators present concurring therein), '
 '</p><br><p></p><br><p>SECTION 1. SENATE ADVICE AND CONSENT SUBJECT TO '
 'DECLARATIONS AND A CONDITION </p><br><br><p>\tThe Senate advises and '
 'consents to the ratification of the Amendment to the Montreal Protocol on '
 'Substances that Deplete the Ozone Layer (the “Montreal Protocol”), adopted '
 'at Kigali on October 15, 2016, by the Twenty-Eighth Meeting of the Parties '
 'to the Montreal Protocol (“The Kigali Amendment”) (Treaty Doc. 117-1), '
 'subject to the declarations of section 2 and the condition of section 3. '
 '</p><br><p></p><br><p>SECTION 2. DECLARATIO

## 3. Retrieving Data on Members of Congress

### Get Members of Congress

Get a list of members of congress as well as their provided information.

In [52]:
def get_members_of_congress(limit=2):
    endpoint = f'member/?api_key={API_KEY}&format=json&limit={limit}'
    response = requests.get(BASE_URL + endpoint)
    if response.status_code == 200:
        members = response.json()['members']
        for member in members:
            print(f"Name: {member['name']}")
            print(f"Party: {member['partyName']}")
            print(f"State: {member['state']}")
            print(f"District: {member['district']}")
            print(f"Terms: {member['terms']}")
            print(f"Bioguide ID: {member['bioguideId']}")
            print()
    else:
        print("Failed to fetch members of Congress.")

print("Members of Congress:")
# get_members_of_congress(limit=num)
# Limit is optional, default is 2 with a max of 250
get_members_of_congress()

Members of Congress:
Name: Nowak, Henry
Party: Democratic
State: New York
District: 33
Terms: {'item': [{'chamber': 'House of Representatives', 'endYear': 1993, 'startYear': 1975}]}
Bioguide ID: N000163

Name: Pascrell, Bill
Party: Democratic
State: New Jersey
District: 9
Terms: {'item': [{'chamber': 'House of Representatives', 'endYear': 2024, 'startYear': 1997}]}
Bioguide ID: P000096



### Get Members of Congress by State

Get a list of members of congress by state as well as their provided information.

Requires a specific state abbreviation and a "True" or "False" to determine whether to get current or all members of that state.

In [53]:
def get_members_of_congress(state, current, limit=250):
    endpoint = f'member/{state}?api_key={API_KEY}&format=json&limit={limit}'
    response = requests.get(BASE_URL + endpoint)
    if response.status_code == 200:
        members = response.json()['members']
        for member in members:
            if current and 'endYear' in member['terms']['item'][0]:
                continue    
            print(f"Name: {member['name']}")
            print(f"Party: {member['partyName']}")
            print(f"State: {member['state']}")
            if 'district' in member:
                print(f"District: {member['district']}")
            else:
                print(f"District: N/A")
            print(f"Terms: {member['terms']}")
            print(f"Bioguide ID: {member['bioguideId']}")
            print()
    else:
        print("Failed to fetch members of Congress.")

print("Members of Congress:")
# get_members_of_congress('StateCode', bool, limit=num)
# bool = True for current members, False for former and current members
# Limit is optional, default is 250 to get all members
get_members_of_congress('AL', True)

Members of Congress:
Name: Strong, Dale W.
Party: Republican
State: Alabama
District: 5
Terms: {'item': [{'chamber': 'House of Representatives', 'startYear': 2023}]}
Bioguide ID: S001220

Name: Carl, Jerry L.
Party: Republican
State: Alabama
District: 1
Terms: {'item': [{'chamber': 'House of Representatives', 'startYear': 2021}]}
Bioguide ID: C001054

Name: Moore, Barry
Party: Republican
State: Alabama
District: 2
Terms: {'item': [{'chamber': 'House of Representatives', 'startYear': 2021}]}
Bioguide ID: M001212

Name: Palmer, Gary J.
Party: Republican
State: Alabama
District: 6
Terms: {'item': [{'chamber': 'House of Representatives', 'startYear': 2015}]}
Bioguide ID: P000609

Name: Sewell, Terri A.
Party: Democratic
State: Alabama
District: 7
Terms: {'item': [{'chamber': 'House of Representatives', 'startYear': 2011}]}
Bioguide ID: S001185

Name: Aderholt, Robert B.
Party: Republican
State: Alabama
District: 4
Terms: {'item': [{'chamber': 'House of Representatives', 'startYear': 1997}]

### Get Legislation Sponsored and Cosponsored by a Member of Congress

Get a list of legislation sponsored and cosponsored by a member of congress as well as their provided information.

Requires a specific congress number, a state abbreviation, and a specific member's bioguide ID.

In [55]:
def get_members_by_state_and_district(congress, state, district, limit=2):
    bioguide_id = ''
    endpoint = f'member/congress/{congress}/{state}/{district}?api_key={API_KEY}&format=json&limit={limit}'
    response = requests.get(BASE_URL + endpoint)
    if response.status_code == 200:
        members = response.json()['members']
        for member in members:
            print(f"Name: {member['name']}")
            print(f"Party: {member['partyName']}")
            print(f"State: {member['state']}")
            print(f"District: {member['district']}")
            print(f"Bioguide ID: {member['bioguideId']}")
            bioguide_id = member['bioguideId']
            print()
    else:
        print("Failed to fetch members by state and district.")

    endpoint = f'member/{bioguide_id}/sponsored-legislation?api_key={API_KEY}&format=json&limit={limit}'
    response = requests.get(BASE_URL + endpoint)
    if response.status_code == 200:
        bills = response.json()['sponsoredLegislation']
        print("Legislation by Member:")
        for bill in bills:
            print("Sponsored")
            print(f"Title: {bill['title']}")
            print(f"Action Date: {bill['latestAction']['actionDate']}")
            print(f"Latest Action: {bill['latestAction']['text']}")
            print(f"Number: {bill['number']}")
            print()
    else:
        print("Failed to fetch legislation by member.")

    endpoint = f'member/{bioguide_id}/cosponsored-legislation?api_key={API_KEY}&format=json&limit={limit}'
    response = requests.get(BASE_URL + endpoint)
    if response.status_code == 200:
        bills = response.json()['cosponsoredLegislation']
        for bill in bills:
            print("Cosponsored")
            print(f"Title: {bill['title']}")
            print(f"Action Date: {bill['latestAction']['actionDate']}")
            print(f"Latest Action: {bill['latestAction']['text']}")
            print(f"Number: {bill['number']}")
            print()
    else:
        print("Failed to fetch cosponsored legislation by member.")
    print()

print("Members by State and District:")
# get_members_by_state_and_district('CongressNumber', 'StateCode', 'DistrictNumber', limit=num)
# Limit is optional, default is 2 with a max of 250
get_members_by_state_and_district('118', 'AL', '4')
get_members_by_state_and_district('118', 'AL', '7')

Members by State and District:
Name: Aderholt, Robert B.
Party: Republican
State: Alabama
District: 4
Bioguide ID: A000055

Legislation by Member:
Sponsored
Title: Deliver for Democracy Act
Action Date: 2024-07-22
Latest Action: Referred to the House Committee on Oversight and Accountability.
Number: 9078

Sponsored
Title: Departments of Labor, Health and Human Services, and Education, and Related Agencies Appropriations Act, 2025
Action Date: 2024-07-12
Latest Action: Placed on the Union Calendar, Calendar No. 485.
Number: 9029

Cosponsored
Title: Celebrating the principles of democracy, religious pluralism, human rights, and the rule of law shared by both the United States and India, the strong people-to-people ties between the United States and India, and the success of the Indian diaspora in the United States.
Action Date: 2024-07-30
Latest Action: Referred to the House Committee on Foreign Affairs.
Number: 1394

Cosponsored
Title: VALID Act
Action Date: 2024-04-11
Latest Action: R